In [1]:
import sail.core
import json
import getpass

password = "SailPassword@123"

sail.core.login("nadams@mghl.com", password, 8000, "52.142.43.247")
print(sail.core.get_datasets())
print(sail.core.get_digital_contracts())

{
    "c44a1520-e407-4e8e-a6cc-7d412ff612e7": {
        "keywords": "tags",
        "tables": [
            {
                "description": "Table containing the BMI, PD-L1, PD-L2 and PD1 levels of patients given age and patient index",
                "compressed_data_size_in_bytes": 14451,
                "all_column_properties": [
                    {
                        "description": "EDIT ME <This is the description of a column containing generic text>",
                        "type": "string",
                        "tags": "EDIT ME <#Column>",
                        "name": "PD1 level before treatment",
                        "units": "GenericString",
                        "id": "e46cac29-e81d-46cf-b244-d5fc4891d4bf"
                    },
                    {
                        "description": "EDIT ME <This is the description of a column containing generic text>",
                        "type": "string",
                        "tags": "EDIT ME <#Column>",
 

In [2]:
#########################################################
# Allocate our job and load our SafeObjects
#########################################################

sail.core.load_safe_objects("SafeObjects")
dg_test_object_guid = "479F9891BBDE41C2B8A06CF2E29F32AC"
dg_increment_safefunction_guid = "473001DB64AF456E971296418B6194FE"
dg_sum_column_guid = "442D0290DEA14CE4B70FB31B891F20B6"

job_id = sail.core.run_job(dg_test_object_guid)
print("Job ID: {} ".format(job_id))

Job ID: 513ce649-ffb6-47d6-8e9b-fd0bdaf66140 


In [3]:
#########################################################
# Extract our identifiers
#########################################################
json_datasets = json.loads(sail.core.get_datasets())
json_digital_contracts = json.loads(sail.core.get_digital_contracts())

user_dataset_name = input("Enter name of Dataset you uploaded:" ) or "SanityDataset"
print("The dataset you wish to access is: {}".format(user_dataset_name))

provision_dc = ""
provision_ds = ""
my_table_guid = ""
for dataset in json_datasets:
    if(json_datasets[dataset]["name"] == user_dataset_name):
        provision_ds = dataset.lower()
        my_table_guid = json_datasets[dataset]["tables"][0]["id"]

for digital_contract in json_digital_contracts:
    if (json_digital_contracts[digital_contract]["dataset"]["id"] == provision_ds):
        provision_dc = digital_contract.lower()
    
print("DC {} DS {} TABLE {}".format(provision_dc, provision_ds, my_table_guid))

Enter name of Dataset you uploaded:IGR Dataset #1
The dataset you wish to access is: IGR Dataset #1
DC 7987505a-451f-4eb3-a12a-b0e53140854a DS 8b754d22-1978-4ede-a85d-97036f588b4a TABLE 091aaa26-e009-4c57-a69e-6785e48fb4e0


In [4]:
job_status = sail.core.get_job_status(job_id)
print("{}:{}".format(job_id,job_status))
#job_status = sail.core.get_job_status(second_job_id)
#print("{}:{}".format(second_job_id,job_status))

513ce649-ffb6-47d6-8e9b-fd0bdaf66140:Missing Input Parameters


In [5]:
first_provision = sail.core.provision_secure_computational_node("orch-test03", provision_dc, provision_ds, "STANDARD_B4MS")

print("Provision ID: {}".format(first_provision))



Provision ID: {
    "Status": true,
    "SCNGuid": "fa6bef17-a5f6-4eee-8279-d3f6c42ea0fb"
}


In [7]:
print(sail.core.wait_for_all_secure_nodes_to_be_provisioned(45000))

{
    "Failed": {

    },
    "Succeeded": {
        "fa6bef17-a5f6-4eee-8279-d3f6c42ea0fb": ""
    },
    "InProgress": {

    },
    "AllDone": true
}


In [8]:
############################################################################
#
# Set the parameters for the job and set a variable for its output
#
############################################################################

set_ds_response = sail.core.set_parameter(job_id, "4DC7BEC15B384FBD9BD721B7BF36A984", my_table_guid)
#set_ds_response = sail.core.set_parameter(second_job_id, "4C8041B4513A44FAA5409662E014519C", provision_ds)
data_id = "{}.{}".format(job_id, "52A2698D65C44433BEDDFEE508C9065F")
print("First job IP {}".format(sail.core.get_ip_for_job(job_id)))
#print("Second job IP {}".format(sail.core.get_ip_for_job(second_job_id)))
job_status = sail.core.get_job_status(job_id)
print("{}:{}".format(job_id,job_status))

First job IP 20.232.177.213
513ce649-ffb6-47d6-8e9b-fd0bdaf66140:Waiting on SCN to report status


In [9]:
data_id = "{}.{}".format(job_id, "52A2698D65C44433BEDDFEE508C9065F")
print(data_id)
pull_data_result = sail.core.pull_data(data_id)
print(pull_data_result)

513ce649-ffb6-47d6-8e9b-fd0bdaf66140.52A2698D65C44433BEDDFEE508C9065F
Success


In [10]:
############################################################################
#
# Produce data frame from CSV
#
############################################################################

import base64
import pickle

message_response = sail.core.wait_for_data(10000)
data_frame = None;
print("hwllo", message_response)
while None != message_response:
    if ( message_response is not None):
        message_json = json.loads(message_response)
        if ( message_json["SignalType"] == 0):
            print("Job {} is running".format(message_json["JobUuid"]))

        if ( message_json["SignalType"] == 1):
            print("Job {} is done".format(message_json["JobUuid"]))

        if ( message_json["SignalType"] == 3):
            data_value = message_json["FileData"]
            decoded_value = base64.b64decode(data_value)
            un_serialized_value = pickle.loads(decoded_value)
            print("Dataframe has been processed from CSV")
            data_frame = un_serialized_value
            
    message_response = sail.core.wait_for_data(2000)


hwllo {
    "JobUuid": "513ce649-ffb6-47d6-8e9b-fd0bdaf66140",
    "SignalType": 0
}
Job 513ce649-ffb6-47d6-8e9b-fd0bdaf66140 is running
Job 513ce649-ffb6-47d6-8e9b-fd0bdaf66140 is done
Dataframe has been processed from CSV


In [11]:
print(data_frame)

     Columns  AGE   BMI  PD-L1 level before treatment  \
0          0   47  27.7                         85.39   
1          1   59  24.6                         84.74   
2          2   35  26.9                         84.94   
3          3   55  31.3                         82.64   
4          4   43  19.9                         86.38   
..       ...  ...   ...                           ...   
295      295   44  38.8                         85.91   
296      296   48  22.8                         81.06   
297      297   78  25.3                         84.71   
298      298   60  18.5                         81.37   
299      299   57  19.5                         89.10   

     PD-L1 level after treatment  PD-L2 level before treatment  \
0                          21.93                         32.14   
1                          37.72                         31.54   
2                          25.96                         32.57   
3                          22.53                   

In [12]:
# Job should be running at this point
import base64
import pickle

message_response = sail.core.wait_for_data(20000)
increment_result = 0;
while None != message_response:
    if ( message_response is not None):
        message_json = json.loads(message_response)
        if ( message_json["SignalType"] == 3):
            data_value = message_json["FileData"]
            decoded_value = base64.b64decode(data_value)
            un_serialized_value = pickle.loads(decoded_value)
            print("The sum of column {} is {}".format(column_name, un_serialized_value))
            
    message_response = sail.core.wait_for_data(5000)

In [ ]:
sail.core.deprovision_secure_computational_node('8a88b5bb-bc55-4afe-bb57-2d5ef395e31d')

In [ ]:
sail.core.exit_current_session()